In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 0.25
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.linewidth'] = 0.25

An upsampler is just a function that we use to upsample an internal (low-resolution) convolutional signal to an external (high-resolution) coordinate system.  For modern convnets that use (n-1)/2 padding, this is just the same as corner-to-corner scaling.  However, if you use a convolutional kernel with different padding, the upsampler can align the grid to make sure that the center of the receptive field is mapped to the correct location on the image.

For example, in the below, we create an AlexNet (that uses zero padding), then we create an upsampler that correctly upsamples the conv5 layer.

In [ ]:
import torchvision
import torch.hub

model = torchvision.models.alexnet(num_classes=365)
url = 'http://gandissect.csail.mit.edu/models/alexnet_places365-6d3c0e75.pth'
sd = torch.hub.load_state_dict_from_url(url)['state_dict']
sd = {k.replace('.module', ''): v for k, v in sd.items()}
model.load_state_dict(sd)

from netdissect import nethook
model = nethook.InstrumentedModel(model)
model = model.cuda()
model.model

model is an instrumented AlexNet whose structure can be seen above.

In [ ]:
from importlib import reload
from netdissect import parallelfolder, renormalize
from torchvision import transforms
from urllib.request import urlopen

reload(parallelfolder)

center_crop = transforms.Compose([
        transforms.Resize((256 ,256)),
        transforms.CenterCrop(227),
        transforms.ToTensor(),
        renormalize.NORMALIZER['imagenet']
])

dataset = parallelfolder.ParallelImageFolders(
    ['dataset/places/val'], transform=[center_crop],
    shuffle=True)


synset_url = 'http://gandissect.csail.mit.edu/models/categories_places365.txt'
classlabels = [r.split(' ')[0][3:] for r in urlopen(synset_url).read().decode('utf-8').split('\n')]


now we have loaded some data, that we will center-crop to 227x227

In [ ]:
from netdissect import renormalize, show
from matplotlib import cm

indices = [300]
model.retain_layer(('features.10', 'conv5'))
batch = torch.cat([dataset[i][0][None,...] for i in indices])
imgs = [renormalize.as_image(t, source=dataset) for t in batch]
preds = model(batch.cuda()).max(1)[1]
prednames = [classlabels[p.item()] for p in preds]
show.blocks([[im], name] for im, name in zip(imgs, prednames))

Above we run the model on a small batch (just one image, image number 300

In [ ]:
from netdissect import upsample
from torch.nn import MaxPool2d
import PIL
reload(upsample)

acts = model.retained_layer('conv5').cpu()

# Define the upsampler function.
upfn = upsample.upsampler(
    (56, 56),                     # The target output shape
    acts.shape[2:],               # The source data shape
    input_shape=batch.shape[2:],  # The actual input data was this shape
    convolutions=list(model.model.features.children())[:10]  # The conv modules from input to data reveal strides
)

upacts = upfn(acts, mode='nearest')
show.blocks(
    [PIL.Image.fromarray((cm.hot(a/(1e-10 +a.max())) * 255).astype('uint8'))]
    for a in upacts[0]
)


Above we upsample the activation to 56x56 (quarter resolution).

In [ ]:
from netdissect import imgviz
reload(imgviz)
iv = imgviz.ImageVisualizer((56, 56), source=dataset, convolutions=list(model.model.features.children())[:10])
show.blocks(
    [[[iv.masked_image(batch[0], acts, (0, u))],
      [iv.heatmap(acts, (0, u), mode='nearest')]] for u in range(acts.shape[1])]
)